<a href="https://colab.research.google.com/github/Borevi4/Borevi4/blob/main/HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import numpy as np
import pandas as pd

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

movies_with_ratings = movies.merge(ratings, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.movieId,
    'rating': movies_with_ratings.rating.astype(int)
})

dataset = dataset[(dataset['rating']>=4)] # только положтельные оценки берем

dataset = pd.pivot_table(dataset, values=['rating'], index=['uid'], columns=['iid'], fill_value=0)

dataset = dataset.droplevel(0, axis=1)

In [59]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [61]:
dataset.head()

iid,1,2,3,5,6,7,8,9,10,11,...,188751,188797,188833,189333,190209,191005,193571,193573,193581,193609
uid,,,,,,,,,,,,,,,,,,,,,
1,4,0,4,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
! pip install lightfm

In [62]:
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

train, test = random_train_test_split(dataset.astype(pd.SparseDtype("int32",0)).sparse.to_coo().tocsr(), 0.3, 42)

print(train.shape, test.shape)

(609, 6298) (609, 6298)


In [63]:
model = LightFM(learning_rate=0.02, loss='warp')
model.fit(train, epochs=80, num_threads=4)

In [64]:
train_precision = precision_at_k(model, train, k=3).mean()
test_precision = precision_at_k(model, test, k=3, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))

Precision: train 0.38, test 0.25.


In [65]:
def sample_recommendation(model, data, user_ids):
    n_users, n_items = data.shape
    for user_id in user_ids:
        known_positives = dataset.loc[user_id,][dataset.loc[user_id,].apply(lambda x: x>0)].index.tolist()

        scores = model.predict(user_id, np.arange(n_items))

        top_items = dataset.columns[np.argsort(-scores)]

        print("User %s" % user_id)
        print("     Known positives:")

        for x in known_positives[:3]:
            print("        %s" % movies[movies.movieId==x].title.iloc[0])

        print("     Recommended:")

        for x in top_items[:3]:
            print("        %s" % movies[movies.movieId==x].title.iloc[0])

sample_recommendation(model, train, [10, 25, 451, 1, 2, 3])

User 10
     Known positives:
        Aladdin (1992)
        Casablanca (1942)
        Mulan (1998)
     Recommended:
        Forrest Gump (1994)
        Braveheart (1995)
        Shawshank Redemption, The (1994)
User 25
     Known positives:
        Dumb & Dumber (Dumb and Dumber) (1994)
        Star Wars: Episode IV - A New Hope (1977)
        Schindler's List (1993)
     Recommended:
        Forrest Gump (1994)
        Shawshank Redemption, The (1994)
        Silence of the Lambs, The (1991)
User 451
     Known positives:
        Toy Story (1995)
        Heat (1995)
        Sense and Sensibility (1995)
     Recommended:
        Star Wars: Episode IV - A New Hope (1977)
        Star Wars: Episode V - The Empire Strikes Back (1980)
        Terminator 2: Judgment Day (1991)
User 1
     Known positives:
        Toy Story (1995)
        Grumpier Old Men (1995)
        Heat (1995)
     Recommended:
        Matrix, The (1999)
        Fight Club (1999)
        Lord of the Rings: The Fellows